In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

from pairing import Reader, Extractor
import definition
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, confusion_matrix

## Data Loading and Preprocessing

In [2]:
train_ratio = 0.6

raw_data = Reader.read_file(definition.DATA_PAIRED_SAMPLE)
index = np.arange(len(raw_data))
np.random.shuffle(index)
index_train = index[:int(train_ratio * len(raw_data))]
index_test = index[int(train_ratio * len(raw_data)):]
raw_data_train = np.array(raw_data)[index_train].tolist()
raw_data_test = np.array(raw_data)[index_test].tolist()

data_train = Extractor.extract_data(raw_data_train)
data_test = Extractor.extract_data(raw_data_test)

100%|███████████████████████████████████████████████████████████████████████████| 1080/1080 [00:00<00:00, 43234.06it/s]


In [3]:
data_train.head()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
0,1,1,5,1,11,2,1,2,1
1,7,7,5,1,9,1,1,8,0
2,4,5,7,1,11,2,7,2,0
3,1,1,7,1,9,1,7,8,1
4,2,3,9,1,22,2,3,0,0


In [4]:
data_train.describe()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
count,10997.000000,10997.000000,10997.000000,10997.000000,10997.000000,10997.000000,10997.000000,10997.000000,10997.000000
mean,7.915431,8.251432,7.902246,1.319633,8.196326,1.486951,10.459034,11.443576,0.319724
std,9.088053,9.098686,3.514481,0.568154,4.240337,0.692643,12.291775,12.249610,0.466391
min,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,5.000000,1.000000,5.000000,1.000000,2.000000,3.000000,0.000000
50%,5.000000,5.000000,8.000000,1.000000,7.000000,1.000000,7.000000,8.000000,0.000000
75%,11.000000,11.000000,10.000000,2.000000,11.000000,2.000000,14.000000,16.000000,1.000000
max,81.000000,81.000000,29.000000,5.000000,29.000000,6.000000,81.000000,82.000000,1.000000


In [5]:
X_train = data_train.drop('target', axis=1)
X_test = data_test.drop('target', axis=1)
y_train = data_train['target']
y_test = data_test['target']

## Training

In [6]:
model = LGBMClassifier()

In [7]:
model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
pred = model.predict(X_test)

In [9]:
f1_score(y_test, pred)

0.8647948164146868

In [12]:
pred_d = [1]
true_d = [0]
confusion_matrix(pred_d, true_d, [1,0])

array([[0, 1],
       [0, 0]], dtype=int64)